In [1]:
import argparse
import os
import copy

import numpy as np
import h5py
import PIL.Image as pil_image
import argparse
import glob 

import torch
from torch import nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from tqdm import tqdm

In [4]:
class FSRCNN(nn.Module):
    def __init__(self, scale_factor, d = 56, s = 12, m = 4):
        super(FSRCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.conv3 = nn.Conv2d(s, m, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(m, s, kernel_size=1)
        self.conv5 = nn.Conv2d(s, d, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(d, 3 * (scale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)
        
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.relu(self.conv4(x))
        x = nn.functional.relu(self.conv5(x))
        x = self.conv6(x)
        x = self.pixel_shuffle(x)
        return x

In [5]:
class TrainFSRCNN(object):
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, n_epochs):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.n_epochs = n_epochs

    def train(self):
        for epoch in range(self.n_epochs):
            running_loss = 0.0
            for i, data in enumerate(self.train_loader, 0):
                inputs, labels = data
                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

            # Validate on the validation set
            val_loss = self.validate()

            # Print statistics
            print("Epoch: %d, Loss: %.3f, Validation Loss: %.3f" %
                  (epoch + 1, running_loss / len(self.train_loader), val_loss))

    def validate(self):
        with torch.no_grad():
            val_loss = 0.0
            for i, data in enumerate(self.val_loader, 0):
                inputs, labels = data
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                val_loss += loss.item()
            return val_loss / len(self.val_loader)

In [22]:
# dataset = datasets.ImageFolder('C:/Users/moreiran/Desktop/FSRCNN_PYTORCH/T91/', transform = transforms.ToTensor())
dataset = datasets.ImageFolder('C:/Users/moreiran/Desktop/FSRCNN_PYTORCH/91-image_x3.h5', transform = transforms.ToTensor())
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)

# Instantiate the TrainFSRCNN class
model = FSRCNN(scale_factor = 2) 
criterion = nn.MSELoss() 
optimizer =  optim.Adam(model.parameters(), lr = 1e-3) 
train_fsrcnn = TrainFSRCNN(model, criterion, optimizer, train_loader, val_loader, n_epochs = 100)

# Start training
train_fsrcnn.train()

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/moreiran/Desktop/FSRCNN_PYTORCH/91-image_x3.h5'